<a href="https://colab.research.google.com/github/ChusUEM/RAG-CHAT/blob/main/load_embedding_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ElasticDocs GPT Blog
# Loading an embedding from Hugging Face into Elasticsearch

This code will show you how to load a supported embedding model from Hugging Face into an elasticsearch cluster in [Elastic Cloud](https://cloud.elastic.co/)

[Blog - ChatGPT and Elasticsearch: OpenAI meets private data](https://www.elastic.co/blog/chatgpt-elasticsearch-openai-meets-private-data)

# Setup


## Install and import required python libraries

Elastic uses the [eland python library](https://github.com/elastic/eland) to download modesl from Hugging Face hub and load them into elasticsearch

In [2]:
pip -q install eland elasticsearch sentence_transformers transformers torch==1.13.1

In [3]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

import getpass

/usr/local/lib/python3.10/dist-packages/eland/ml/_optional.py:116: UserWarning: Eland requires version '1.3' or newer of 'sklearn' (version '1.2.2' currently installed). Use pip or conda to update sklearn.
  warnings.warn(msg, UserWarning)
<ipython-input-3-d329d25d548f>:5: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


## Configure elasticsearch authentication.
The recommended authentication approach is using the [Elastic Cloud ID](https://www.elastic.co/guide/en/cloud/current/ec-cloud-id.html) and a [cluster level API key](https://www.elastic.co/guide/en/kibana/current/api-keys.html)

You can use any method you wish to set the required credentials. We are using getpass in this example to prompt for credentials to avoide storing them in github.

In [4]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID: RAG-CHAT:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRlMzA3OWFlODYzMDM0ODYzYTUzMWI0NWUzMzc4ZTcwOSQzYWZkMWUwMzRiMGU0MWQyYjI0NDA2M2RkNzY0ZTk5Mw==')
es_user = getpass.getpass('Enter cluster username:  elastic')
es_pass = getpass.getpass('Enter cluster password: SHKfveRBWscVnFLrLPG0udwu')


#es_api_id = getpass.getpass('Enter cluster API key ID: RAG-CHAT')
#es_api_key = getpass.getpass('Enter cluster API key: RVVPcUdvNEJkREdmWWJXZWc0Zk86VlJiVUtCblNTYW1Xb2llcTQzUkJlQQ==')

Enter Elastic Cloud ID: RAG-CHAT:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRlMzA3OWFlODYzMDM0ODYzYTUzMWI0NWUzMzc4ZTcwOSQzYWZkMWUwMzRiMGU0MWQyYjI0NDA2M2RkNzY0ZTk5Mw==··········
Enter cluster username:  elastic··········
Enter cluster password: SHKfveRBWscVnFLrLPG0udwu··········


## Connect to Elastic Cloud

In [5]:
'''es = Elasticsearch(cloud_id=es_cloud_id,
                   api_key=(es_api_id, es_api_key)
                   )'''
es = Elasticsearch(cloud_id=es_cloud_id,
                   basic_auth=(es_user, es_pass)
                   )
es.info() # should return cluster info

ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': 'e3079ae863034863a531b45e3378e709', 'cluster_uuid': 'blK9EXzbQ1GN6FtA9xswBg', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Load the model From Hugging Face into Elasticsearch
Here we specify the model id from Hugging Face. The easiest way to get this id is clicking the copy the model name icon next to the name on the model page.

When calling `TransformerModel` you specify the HF model id and the task type. You can try specifying `auto` and eland will attempt to determine the correct type from info in the model config. This is not always possible so a list of specific `task_type` values can be viewed in the following code:
[Supported values](https://github.com/elastic/eland/blob/15a300728876022b206161d71055c67b500a0192/eland/ml/pytorch/transformers.py#*L41*)

In [6]:
import torch

# Inicializar token de Hugging Face (si aún no se ha hecho)
import os
if "HF_TOKEN" not in os.environ:
    from huggingface_hub import notebook_login
    notebook_login()  # Sigue las instrucciones para autenticar

# Set the model name from Hugging Face and task type
hf_model_id='sentence-transformers/all-distilroberta-v1'
tm = TransformerModel(model_id=hf_model_id, task_type='text_embedding')

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


  0%|          | 0/312 [00:00<?, ? parts/s]

# Starting the Model

## View information about the model
This is not required but can be handy to get a model overivew

In [7]:
# List the in elasticsearch
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

{'count': 1,
 'trained_model_configs': [{'model_id': 'sentence-transformers__all-distilroberta-v1',
   'model_type': 'pytorch',
   'created_by': 'api_user',
   'version': '12.0.0',
   'create_time': 1709921099186,
   'model_size_bytes': 0,
   'estimated_operations': 0,
   'license_level': 'platinum',
   'description': "Model sentence-transformers/all-distilroberta-v1 for task type 'text_embedding'",
   'tags': [],
   'metadata': {'per_allocation_memory_bytes': 322574444,
    'per_deployment_memory_bytes': 326119456},
   'input': {'field_names': ['text_field']},
   'inference_config': {'text_embedding': {'vocabulary': {'index': '.ml-inference-native-000002'},
     'tokenization': {'roberta': {'do_lower_case': False,
       'with_special_tokens': True,
       'max_sequence_length': 512,
       'truncate': 'first',
       'span': -1,
       'add_prefix_space': False}}}},
   'location': {'index': {'name': '.ml-inference-native-000002'}}}]}

## Deploy the model
This will load the model on the ML nodes and start the process(es) making it available for the NLP task

In [8]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

{'assignment': {'task_parameters': {'model_id': 'sentence-transformers__all-distilroberta-v1',
   'deployment_id': 'sentence-transformers__all-distilroberta-v1',
   'model_bytes': 326153640,
   'threads_per_allocation': 1,
   'number_of_allocations': 1,
   'queue_capacity': 1024,
   'cache_size': '326153640b',
   'priority': 'normal',
   'per_deployment_memory_bytes': 326119456,
   'per_allocation_memory_bytes': 322574444},
  'routing_table': {'l55QBB9vTKmZhYIZ6qljcQ': {'current_allocations': 1,
    'target_allocations': 1,
    'routing_state': 'started',
    'reason': ''}},
  'assignment_state': 'started',
  'start_time': '2024-03-08T18:06:26.476981063Z',
  'max_assigned_allocations': 1}}

## Verify the model started without issue
Should output -> {'routing_state': 'started'}

In [9]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']

{'routing_state': 'started'}